In [1]:
from collections import Counter

import pysam
import pybedtools
from Bio import SeqIO
import pandas as pd
import os
from IPython.core.display import HTML
from Bio.Seq import Seq

4 cases 

is read 1 and is not reverse:

    start of read one is start of cDNA

is read 1 and is reverse:

    end of read 1 is start of cDNA

is read 2 and is not reverse:

    start of read 2 is end of cDNA
    
is read 2 and is reverse:

    end of read2 is end of cDNA


In [11]:
from collections import Counter

In [47]:
def filer_short_reads(interval):
    return len(interval) == 21

def get_seq_logos(fn, out_file):
    hg19_chrom_sizes = "/projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes"
    hg19_fasta = "/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa"
    
    sam_file = pysam.Samfile(fn)
    
    cDNA_start_neg = []
    cDNA_start_pos = []
    cDNA_end_neg = []
    cDNA_end_pos = []
    
    read_1_start_base = Counter()
    read_2_start_base = Counter()
    for x, read in enumerate(sam_file):
        if read.is_read1 and not read.is_reverse:
            cDNA_start_neg.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid),
                                                                    read.positions[0],
                                                                    read.positions[0] + 1,
                                                                   0,
                                                                   0,
                                                                   "-"]))
            r1_seq = Seq(read.query)
            read_1_start_base[r1_seq[0]] += 1

        if read.is_read1 and read.is_reverse:
            cDNA_start_pos.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid),
                                                                    read.positions[-1],
                                                                    read.positions[-1] + 1,
                                                                   0,
                                                                   0,
                                                                   "+"]))
            r1_seq = Seq(read.query).reverse_complement()
            read_1_start_base[r1_seq[0]] += 1

        if read.is_read2 and not read.is_reverse:
            cDNA_end_pos.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid),
                                                                  read.positions[0],
                                                                  read.positions[0] + 1,
                                                                 0,
                                                                   0,
                                                                   "+"]))
            r2_seq = Seq(read.query)
            read_2_start_base[r2_seq[0]] += 1

        if read.is_read2 and read.is_reverse:
            cDNA_end_neg.append(pybedtools.create_interval_from_list([sam_file.getrname(read.tid), 
                                                                  read.positions[-1], 
                                                                  read.positions[-1] + 1,
                                                                 0,
                                                                   0,
                                                                   "-"]))
            r2_seq = Seq(read.query).reverse_complement()
            read_2_start_base[r2_seq[0]] += 1

    print "done getting reads"
    cDNA_start = pybedtools.BedTool(cDNA_start_pos + cDNA_start_neg)
    cDNA_end = pybedtools.BedTool(cDNA_end_pos + cDNA_end_neg)
        
    #Expand out to 10bp around the cDNA start
    cDNA_start = cDNA_start.slop(b=10, g=hg19_chrom_sizes)
    #Remove reads that are near the start and end of a chromosome and will therefore mess with seqlogo
    cDNA_start = cDNA_start.filter(filer_short_reads).saveas()
    result = cDNA_start.sequence(fi=hg19_fasta, s=True, fo="cDNA_start.fasta")
    
    #Expand out to 10bp around the cDNA start
    cDNA_end = cDNA_end.slop(b=10, g=hg19_chrom_sizes)
    #Remove reads that are near the start and end of a chromosome and will therefore mess with seqlogo
    cDNA_end = cDNA_end.filter(filer_short_reads).saveas()
    result = cDNA_end.sequence(fi=hg19_fasta, s=True, fo="cDNA_end.fasta")

    out_cDNA_start = out_file + ".cDNA_start.pdf"
    out_cDNA_end = out_file + ".cDNA_end.pdf"
    
#     !seqlogo -f cDNA_start.fasta -F PDF -Y -a -S -n -x "nt from start" -s -10 -c -k 1 -B 2 -T 1  > $out_cDNA_start
#     !seqlogo -f cDNA_end.fasta -F PDF -Y -a -S -n -x "nt from end" -s -10 -c -k 1 -B 2 -T 1  > $out_cDNA_end
    return read_1_start_base, read_2_start_base

In [22]:
#PTBP Rep 2
file_dir = "/home/gpratt/projects/encode/analysis/ad-hoc/encode_bams"
get_seq_logos(os.path.join(file_dir, "ENCFF511GIO.bam"), "PTBP_K562_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF227OUZ.bam"), "PTBP_K562_Rep2")            
get_seq_logos(os.path.join(file_dir, "ENCFF289RXU.bam"), "U2AF2_K562_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF995HAG.bam"), "U2AF2_K562_Rep2")
get_seq_logos(os.path.join(file_dir, "ENCFF523UFV.bam"), "U2AF2_HepG2_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF728QXQ.bam"), "U2AF2_HepG2_Rep2")
get_seq_logos(os.path.join(file_dir, "ENCFF282IRT.bam"), "TRA2A_K562_Rep1")
get_seq_logos(os.path.join(file_dir, "ENCFF111LPA.bam"), "TRA2A_K562_Rep2")

done getting reads
done getting reads
done getting reads
done getting reads
done getting reads


IOError: file `/home/gpratt/projects/encode/analysis/ad-hoc/encode_bams/ENCFF728QXQ.bam` not found

In [31]:
os.path.join(file_dir, "TDP_AFFSC_TARDBP.merged.bam")

'/projects/ps-yeolab3/encode/analysis/encode_master/TDP_AFFSC_TARDBP.merged.bam'

In [30]:
file_dir = "/projects/ps-yeolab3/encode/analysis/encode_master/"

for read in pysam.Samfile(os.path.join(file_dir, "TDP_AFFSC_TARDBP.merged.bam")):
    if read.is_reverse:
        print read
        break

GACTAGCCCC:K00180:248:HCT55BBXX:7:1223:8308:48034	147	0	14943	255	20M9D23M	0	14932	43	GGGCGGGCAGCGGCCCTGCCTGCGCCTCATGACCAGCTTGTTG	array('B', [41, 27, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 37, 37, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 37, 41, 41, 41, 41, 41, 41])	[('NH', 1), ('HI', 1), ('AS', 53), ('nM', 2), ('NM', 9), ('MD', '20^TCCTACCCT23'), ('jM', array('b', [-1])), ('jI', array('i', [-1])), ('RG', 'foo')]


In [72]:
read.seq.upper()

Seq('TGGGCTATTTAGGCTTTATGG', SingleLetterAlphabet())

In [73]:
result = defaultdict(Counter)

for read in SeqIO.parse("cDNA_end.fasta", "fasta"):
    for loc, base in enumerate(read.seq.upper()):
        result[loc][base] += 1
result = pd.DataFrame(result)

In [75]:
HTML(result.to_html())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
A,301379,297298,300712,305817,301459,299745,297147,299747,262917,218200,252120,256013,299506,321799,322639,298689,285403,282459,277169,276587,271759
C,288409,291780,282503,282261,286119,285956,287544,298892,264370,212057,225436,276117,282791,350645,323652,295166,306999,320121,328624,317128,319186
G,307171,300888,309343,300849,307584,291796,298093,255276,257345,243812,323407,307351,340599,284698,282216,343836,354318,353578,340527,351426,360143
N,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
T,318995,325988,323396,327027,320792,338457,333170,362039,431322,541885,414991,376473,293058,258812,287447,278263,269234,259796,269634,270813,264866


In [48]:
r1_start, r2_start = get_seq_logos(os.path.join(file_dir, "TDP_AFFSC_TARDBP.merged.bam"), "TDP_AFFSC_TARDBP")


done getting reads


In [49]:
foo = pd.DataFrame({"r1": r1_start, "r2": r2_start})

In [50]:
bar = foo / foo.sum()

In [51]:
foo.sum()

r1    1215956
r2    1215956
dtype: int64

In [52]:
foo

,r1,r2
A,335211,251048
C,202907,221790
G,307840,324043
N,3,9
T,369995,419066


In [53]:
bar

,r1,r2
A,0.275677,0.206461
C,0.166870,0.182400
G,0.253167,0.266492
N,0.000002,0.000007
T,0.304283,0.344639


In [7]:
file_dir = "/projects/ps-yeolab3/encode/analysis/encode_master/"

get_seq_logos(os.path.join(file_dir, "TDP_AFFSC_TARDBP.merged.bam"), "TDP_AFFSC_TARDBP")
get_seq_logos(os.path.join(file_dir, "TDP_SS4_MN_TARDBP.merged.bam"), "TDP_SS4_MN_TARDBP")
get_seq_logos(os.path.join(file_dir, "AS_TDP_SSIII_IN_S112_L008_R1_001.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "AS_TDP_SSIII_IN_S112_L008_R1_001")
get_seq_logos(os.path.join(file_dir, "AS_TDP_AFFSC_IN_S108_L008_R1_001.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "AS_TDP_AFFSC_IN_S108_L008_R1_001")

done getting reads
done getting reads
done getting reads
done getting reads


In [23]:
file_dir = "/projects/ps-yeolab2/encode/analysis/encode_v12"
get_seq_logos(os.path.join(file_dir, "254_02_BCCIP.merged.bam"), "254_02_BCCIP")

done getting reads


In [25]:
get_seq_logos(os.path.join(file_dir, "254_INPUT_GAGATTCC-TAATCTTA_L006_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "254_INPUT_BCCIP")

done getting reads


In [ ]:
get_seq_logos(os.path.join(file_dir, "242_INPUT_CGCTCATT-ATAGAGGC_L005_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "242_INPUT_U2AF2")
get_seq_logos(os.path.join(file_dir, "272_INPUT_CGCTCATT-GGCTCTGA_L002_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.bam"), "272_INPUT_U2AF2")